In [85]:
from chordextractor import ChordExtractor
import chordgraph

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
from scipy.stats import itemfreq
import time
import os
import pickle
#from itertools import imap
#from functools import partial
#import math

In [142]:
# Get file names of type type_ in all the subdirectories of the path
def get_filenames(path, type_ = None):
    if type_ == None:
        return sum(map(lambda walks: (map(lambda name: os.path.join(walks[0], name),
                                                                           walks[2])), os.walk(path)), [])
    else:
        return filter(lambda f: f.endswith(type_), sum(map(lambda walks: (map(lambda name: os.path.join(walks[0], name),
                                                                           walks[2])), os.walk(path)), []))
       
# Getting set of all possible chords after normalization        
chord_encode =  {k: i for k, i in zip(chordgraph.ChordGraph().normalized_chords,
                                      range(len(chordgraph.ChordGraph().normalized_chords)))}        
        
# Enumerating chords for processing and storage optimization
def encode_chords(chords, encoder = None, type_ = None):
    if type_ == 'chord':
        return map(lambda chord: encoder[chord], chords)
    return chords

# Generate all sequences of 4 consecutive notes
def get_sequences(data):
    #for seq in imap(lambda i: [data[i], data[i+1], data[i+2], data[i+3]], range(0, len(data)-3)):
    #    yield seq
    return map(lambda i: [data[i], data[i+1], data[i+2], data[i+3]], range(0, len(data)-3))

# Returns index of the note quartet in the vector
def notestoindex(numbers, num):
    return reduce(lambda x, y: num*x+y, numbers)

# Build vector of ocurrences of note quartets
def build_vector(sequence, num):
    vector = [0] * num**4
    seqs_indexed = map(notestoindex, sequence, [num]*len(sequence))
    for x in itemfreq(seqs_indexed):
        vector[x[0]] = x[1]
    return vector

def vector_from_file(filename_):
    extractor = ChordExtractor(filename_)
    chord_vector = build_vector(get_sequences(encode_chords(extractor.getNormalizedChords(), chord_encode, 'chord')),14)
    pitch_vector = build_vector(get_sequences(encode_chords(extractor.getNormalizedPitches())),12)
    return chord_vector, pitch_vector

#normalizedChords = extractor.getNormalizedChords()
#normalizedPitches = extractor.getNormalizedPitches()
# normalizedDurations = extractor.getChordDurations()

# Build vectors from MIDI files and pickle them
def vectors_from_files(path, class_, count):
    files = map(lambda x: x[0], sorted(map(lambda f: [f, os.stat(f).st_size], get_filenames(path)), 
                                   key = lambda x: x[1]))[:count]
    c = 0
    for name in files:
        with open(str(class_)+str(c)+".dat", "wb") as f:
            pickle.dump(vector_from_file(name), f)
            print name+" done"
            print str(class_)+str(c)+".dat"
            c+=1
    #X = map(lambda name: vector_from_file(name), files)
    return True

# Loading vectors from pickles
def get_vectors_from_pickles(path):
    class0 = filter(lambda x: x.split('/')[-1][0]=='0', get_filenames(path, ".dat"))
    class1 = filter(lambda x: x.split('/')[-1][0]=='1', get_filenames(path, ".dat"))
    vector0 = []
    vector1 = []
    for name in class0:
        with open(name, "rb") as f:
            vector0.append(pickle.load(f))
    for name in class1:
        with open(name, "rb") as f:
            vector1.append(pickle.load(f))
    return vector0, vector1

In [124]:
path_to_class0 = './70s/'
path_to_class1 = './80s/'
# Renaming 50s and 60s .kar to .mid
map1 = map(lambda name: os.rename(name, name.replace(".kar", ".mid").replace(".KAR", ".mid")), 
           get_filenames(path_to_class0))
map2 = map(lambda name: os.rename(name, name.replace(".kar", ".mid").replace(".KAR", ".mid")), 
           get_filenames(path_to_class1))

# Identifying the smallest directory in terms of number of midis
midi_count = min(len(get_filenames(path_to_class0)), len(get_filenames(path_to_class1)))
midi_count

193

In [123]:
start_time = time.time()
vectors0 = vectors_from_files(path_to_class0, 0, 72)
print "Class 0 is done"
print("--- %s seconds ---" % (time.time() - start_time))
vectors1 = vectors_from_files(path_to_class1, 1, 72)
print "Class 1 is done"
print("--- %s seconds ---" % (time.time() - start_time))

./70s/DON MCLEAN Vincent.mid done
050.dat
./70s/JUICE NEWTON Angel of the morning.mid done
051.dat
./70s/CAT STEVENS Morning Has Broken.mid done
052.dat
./70s/ENGLEBERT Am I that easy to forget.mid done
053.dat
./70s/LEONARD COHEN Suzanne.mid done
054.dat
./70s/RUBY & ROMANTICS Our day will come.mid done
055.dat
./70s/BANGLES Eternal Flame.mid done
056.dat
./70s/KANSAS Dust In The Wind.mid done
057.dat
./70s/CARLY SIMON Nobody Does It Better.mid done
058.dat
./70s/BJ THOMAS Raindrops Keep Falling On My Head.mid done
059.dat
./70s/JIM CROCE Time in a bottle.mid done
060.dat
./70s/CAT STEVENS Father And Son.mid done
061.dat
./70s/SAMMY DAVIS JR if i ruled the world.mid done
062.dat
./70s/GLEN CAMPBELL By The Time I Get To Phoenix.mid done
063.dat
./70s/TIFFANY I Think We're Alone Now.mid done
064.dat
./70s/JAY & AMERICANS Come a little bit closer.mid done
065.dat
./70s/TODD RUNDGREN A dream go.mid done
066.dat
./70s/BANGLES Manic Monday.mid done
067.dat
./70s/LEONARD COHEN Jazz police.mi

IndexError: string index out of range

In [143]:
vector0, vector1 = get_vectors_from_pickles('vectors')

In [154]:
len(vector0), len(vector1)

(72, 72)

In [162]:
# extractor = ChordExtractor(fileName="albeniz.mid")
# normalizedChords = extractor.getNormalizedChords()
# normalizedPitches = extractor.getNormalizedPitches()
# normalizedDurations = extractor.getChordDurations()
# type(normalizedDurations)

In [ ]:
# Merging the vectors together and extracting the attributes and the label
X = vectors0[0]+vectors1[0]
Y = vectors0[1]+vectors1[1]

# Splitting the data into training and testing subsets 90 to 10 %
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1, random_state = 25)

# Splitting the training subset into training and validating subsets 80 to 20 %
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, test_size=0.2, random_state = 25)

In [ ]:
print len(X_train), len(Y_train)
print len(X_valid), len(Y_valid)
print len(X_test), len(Y_test)

In [ ]:
# Fitting the parameter C for the SVM model on the training / validation sets
clf = svm.SVC(kernel = 'poly', degree = 2, C=0.9)
clf.fit(X_train, Y_train)
print "We successfully predict the {} percent of training data".format((Y_train==clf.predict(X_train)).sum()*100/len(Y_train))
print "We successfully predict the {} percent of test data".format((Y_test==clf.predict(X_test)).sum()*100/len(Y_test))

In [ ]:
# Function to train SVM on training set and to predict the validation set
def trainpredict(degree, C):
    clf = svm.SVC(kernel = 'poly', degree = degree, C = C) 
    clf.fit(X_train, Y_train)
    return (Y_valid == clf.predict(X_valid)).sum()*100/len(Y_valid), degree, C

# Function to fit poly SVMs of degrees 1 to 4 each with Cs from the list
# [math.exp(i) for i in np.linspace(-10,2*x,200)]
def fitDegree_C():
    return map(lambda degree: map(trainpredict, [degree]*len(np.linspace(-10, 2*degree, 200)),
                            map(lambda c: math.exp(c), np.linspace(-10, 2*degree, 200))), range(1, 5)) 
# Each of the sublists correspond to a degree (1 to 4) and has tuples with (accuracy, degree, C)
fitted = fitDegree_C()

In [ ]:
# print 1.0*np.count_nonzero(map(lambda i: X[i] == vectors0[0][i], range(0, len(vectors0[0]))))/len(vectors0[0])
# print 1.0*np.count_nonzero(map(lambda i: X[i+76] == vectors1[0][i], range(0, len(vectors1[0]))))/len(vectors1[0])
# print 1.0*np.count_nonzero(map(lambda i: Y[i] == vectors0[1][i], range(0, len(vectors0[1]))))/len(vectors0[1])
# print 1.0*np.count_nonzero(map(lambda i: Y[i+76] == vectors1[1][i], range(0, len(vectors1[1]))))/len(vectors1[1])
# #np.count_nonzero([True, True])

In [ ]:
flatten = sum(fitted, [])
# Looking for the optimal parameters
max(enumerate(flatten), key = lambda x: x[1])